Lets load initial dataset.

In [ ]:
%%sh
wget -N https://tomash.s3.eu-west-1.amazonaws.com/artifacts/bank-data.csv

Lets import some libraries and then split source dataset into two sets: training and validation. Validation set will contain 20% of initial source dataset. We will use later validation data set to validate how good (or bad ;) our model is. 

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

source_data = pd.read_csv("./bank-data.csv")
train_data, validation_data = train_test_split(source_data, test_size=0.2)

train_data.to_csv("train.csv", header=True, index=False)
validation_data.to_csv("validation.csv", header=True, index=False)

Installing AutoGluon

In [ ]:
!pip install -U pip
!pip install -U setuptools wheel
!pip install -U "mxnet<2.0.0"
!pip install autogluon

In [ ]:
from autogluon.tabular import TabularDataset, TabularPredictor
train_data = TabularDataset('./train.csv')
train_data.head()

In [ ]:
train_data.count()

In [ ]:
prediction_label = 'y'
print("Summary of class variable: \n", train_data[prediction_label].describe())

In [ ]:
predictor = TabularPredictor(label=prediction_label).fit(train_data)

Lets validate how good (or bad ;) our model actually is. Lets compare validation data againts prediction from that model.

In [ ]:
test_data = TabularDataset('./validation.csv')
y_test = test_data[prediction_label] 
clean_test_data = test_data.drop(columns=[prediction_label])  # delete label column to prove we're not cheating
clean_test_data.head() # Lets see how they look like

In [ ]:
y_pred = predictor.predict(clean_test_data)
print("Predictions:  \n", y_pred)
perf = predictor.evaluate_predictions(y_true=y_test, y_pred=y_pred, auxiliary_metrics=True)

Lets see how different AI models look like

In [ ]:
predictor.leaderboard(test_data, silent=True)

We can see 'probability' for each of class using PREDICT_PROBA method.

In [ ]:
pred_probs = predictor.predict_proba(clean_test_data)
pred_probs.head(5)

Or we can directly get predicted class

In [ ]:
pred_probs = predictor.predict(clean_test_data)
pred_probs.head(5)

Some additional information about our trained models

In [ ]:
results = predictor.fit_summary(show_plot=True)

We can also use specific AI model for our prediction

In [ ]:
predictor.predict(test_data, model='KNeighborsUnif')

In [ ]:
y_pred = predictor.predict(clean_test_data, model='KNeighborsUnif')
print("Predictions:  \n", y_pred)
perf = predictor.evaluate_predictions(y_true=y_test, y_pred=y_pred, auxiliary_metrics=True)